In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-03-26 01:35:04.692581: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-26 01:35:04.692635: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_data = pd.read_csv('raw_data/train.csv')
X_test = pd.read_csv('raw_data/test.csv')

In [3]:
X_tv = train_data['text']
y_tv = train_data['target']

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size = .3, random_state = 42)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [6]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

2022-03-26 01:35:21.175490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-26 01:35:21.175585: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-26 01:35:21.175616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-9OD3U9E): /proc/driver/nvidia/version does not exist
2022-03-26 01:35:21.176966: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TF

In [7]:
X_train.head()

1186    Ashes 2015: AustraliaÛªs collapse at Trent Br...
4071    GREAT MICHIGAN TECHNIQUE CAMP\nB1G THANKS TO @...
5461    CNN: Tennessee movie theater shooting suspect ...
5787    Still rioting in a couple of hours left until ...
7445    Crack in the path where I wiped out this morni...
Name: text, dtype: object

In [8]:
X_train[7445]

'Crack in the path where I wiped out this morning during beach run. Surface wounds on left elbow and right knee. http://t.co/yaqRSximph'

In [9]:
X_train = X_train.apply(lambda x : np.array(tokenizer.encode(x)))

In [10]:
X_val = X_val.apply(lambda x : np.array(tokenizer.encode(x)))

In [11]:
X_tv = pad_sequences(X_tv.apply(lambda x : tokenizer.encode(x)), padding = 'post')

In [12]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
es = EarlyStopping(patience = 2, restore_best_weights = True)

In [ ]:
history = model.fit(X_tv, y_tv, callbacks = [es], epochs = 100, validation_split = .2, batch_size = 128)

Epoch 1/100
